# L5b: Single Asset Geometric Brownian Motion Models
In this lecture, we introduce our first continuous-time model for asset prices, the Geometric Brownian Motion (GBM) model. This model is __widely__ used in finance due to its mathematical tractability and ability to capture key features of asset price dynamics.

> __Learning Objectives:__
>
> Fill me in later

Let's get started!
___

## Examples
Today, we will be using the following examples to illustrate key concepts:

___

## Single Asset Geometric Brownian Motion (GMB)
Geometric Brownian motion (GBM) is a stochastic differential equation describing the share price $S(t)$ 
as a random walk with drift and volatility proportional to $S(t)$:
$$
\begin{align*}
\frac{dS\left(t\right)}{S(t)} &= \mu\,{dt}+\sigma\,{dW(t)}\\
\end{align*}
$$
Here, $\mu\in\mathbb{R}$ denotes a constant drift (growth) parameter, $\sigma>0$ is a constant __volatility__ parameter, and $dW$ represents a Wiener noise (randomness) process:

> __Wiener Noise__
> 
> A Wiener Process is a real-valued continuous-time stochastic process named after Norbert Wiener for the study of one-dimensional Brownian motion. A Wiener process is a continuous one-dimensional stochastic process $\left\{W\left(t\right), 0\leq{t}\leq{T}\right\}$ with the following properties:
>
> * The noise vanishes at zero: $W\left(0\right)$ = $0$ with probability $1$
> * The increments $\left\{W(t_{1}) - W(t_{o}),\dots, W(t_{k}) - W(t_{k-1})\right\}$ are independent for any $k$ and $0\leq{t_{o}}< t_{1} < \dots < t_{k} \leq{T}$}
> * The increment $(W(t) - W(s))\sim~\mathcal{N}\left(0,t-s\right)$ for any $0\leq{s}< t \leq{T}$, where $\mathcal{N}\left(0,t-s\right)$ denotes a normally distributed random variable with mean $0$ and variance $t - s$. Thus, we can write $W(t) - W(s) = \left(\sqrt{t-s}\right)\;Z(0,1)$, where $Z(0,1)$ is a standard normal random variable.
> 

We can solve the GBM model using Itô calculus. The GBM model has the analytical solution:
$$
\boxed{
\begin{align*}
S_{t} &= S_{0}\;\exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)\left(t - t_{0}\right) + (\sigma\sqrt{t-t_{0}})\;{Z(0,1)}\Biggr]\\
\end{align*}}
$$
where $S_{\circ}$ is the initial share price at $t_{\circ}$, and $Z(0,1)$ is a standard normal random variable. The expectation and variance of a GBM model is given by:
$$
\begin{align*}
\mathbb{E}\left(S_{t}\right) &= S_{0}\;\exp\left(\mu\Delta{t}\right)\\
\text{Var}\left(S_{t}\right) &= S_{0}^{2}e^{2\mu\cdot\Delta{t}}\left[e^{\sigma^{2}{\Delta{t}}} - 1\right]
\end{align*}
$$

__Want to know where this comes from?__ For a derivation of the GBM model solution, [check out these notes](CHEME-5660-L5b-GBM-Solution-Derivation-Fall-2025.ipynb).

## Estimation of GBM Parameters
We can estimate the parameters drift $\mu$ and volatility $\sigma$ of a GBM model using historical price data.

### Volatility
Let's start with the volatility term $\sigma$. Suppose we price observations from $0\rightarrow{T}$. From these observations, we can construct a time series of length $N-1$ of continuously compounded growth rates $\left\{\mu_1,\mu_2,\ldots,\mu_{N-1}\right\}$ for a stock. The sample standard deviation of the continuously compounded growth rate (our risk measure), is given by:
$$
\boxed{
\begin{align*}
\sigma_{\mu} &= \underbrace{\sqrt{\frac{1}{N - 2}\sum_{t=2}^{N}\left(\mu_t - \mu^{\prime}\right)^2}}_{\text{volatility of the growth rate}} \\
\end{align*}}
$$
where $\mu^{\prime} = \frac{1}{N-1}\sum_{t=2}^{N}\mu_t$ is the sample mean of the $N-1$ growth observations. The denominator $N-2$ represents the degrees of freedom for the sample standard deviation: $(N-1) - 1 = N-2$. We'll typically compute the standard deviation using [the `std(...)` method exported by the `Statistics.jl` package](https://docs.julialang.org/en/v1/stdlib/Statistics/#Statistics.std).



### Drift
Next, let's move on to the drift term $\mu$ (units: inverse years). The drift term $\mu$ represents the expected return of the asset over a given time period. There are two common methods to estimate the drift term $\mu$ of the GBM model.

> __Estimating the Drift Term ($\mu$):__
>
> * __Method 1: Average Log Growth__: The first, is to compute the log growth rates of the asset prices over a historical period and then take the average of these log growth rates. This method is straightforward and provides a simple estimate of the drift.
> * __Method 2: Linear Regression__: The second method involves performing a linear regression on the logarithm of the asset prices over time __assuming the noise term is zero__. By fitting a linear model to the log prices, we can estimate the slope of the line, which corresponds to the drift term $\mu$. This method can be more robust, especially when dealing with noisy data, e.g., we can use robust regression techniques to mitigate the impact of outliers. 
> 
> Let's check out the ideas behind the linear regression method.

#### Linear Regression
Let's assume the we have a price dataset for some firm that we've collected everyday for some time period, i.e., for $t = 0\rightarrow{T}$. We can denote the price at time step $i$ as $S_{i}$, where $i=0,1,2,\ldots,N$ and $N$ is the total number of time steps (e.g., days). The time step size is $\Delta{t}$ (e.g., 1 day). Then, we can write the GMB solution at time $T = i\Delta{t}$ as:
$$
\begin{align*}
S_{i} &= S_{0}\;\exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)\left(i\Delta{t}\right) + (\sigma\sqrt{i\Delta{t}})\;{Z}\Biggr]\quad{i = 1,2,\ldots,N}\\
\end{align*}
$$
In this method, we assume that the noise term is zero, i.e., $\sigma = 0$. This simplifies the GBM solution to:
$$
\begin{align*}
S_{i} &= S_{0}\;\exp\left(\mu\cdot{i\Delta{t}}\right)\quad\Longrightarrow\text{Taking the natural logarithm of both sides gives:}\\
\ln(S_{i}) &= \ln(S_{0}) + \mu\cdot{i\Delta{t}}\quad\quad{i=1,2,\ldots,N}\\
\end{align*}
$$
where $\ln(S_{0})$ is a constant (share price at time $t=0$, y-intercept). This equation represents a linear relationship between $\ln(S_{i})$ (observation) and $i\Delta{t}$ (independent variable), where the slope of the line is $\mu$ (units: inverse years). Given the price observations ${S_{1},\ldots,S_{N}}$, we can construct an __overdetermined__ system of equations:
$$
\begin{align*}
\hat{\mathbf{X}}\mathbf{\theta} + \epsilon &= \mathbf{y}\\
\end{align*}
$$
where $\mathbf{\theta}$ is the parameter vector we want to estimate (the y-intercept $\ln(S_{0})$ and slope $m = \mu$), $\epsilon\sim\mathcal{N}(0,\sigma_{\epsilon}^{2})$ is an error model, $\mathbf{X}$ is the $N\times{2}$ design matrix, and $\mathbf{y}$ is the $N\times{1}$ observation vector. The design matrix $\mathbf{X}$ is given by:
$$
\begin{align*}
\hat{\mathbf{X}} &= \begin{bmatrix}
1 & 1\Delta{t} \\
1 & 2\Delta{t} \\
\vdots & \vdots \\
1 & N\Delta{t}
\end{bmatrix}\\
\end{align*}
$$
while the observation vector $\mathbf{y}$ is given by:
$$
\begin{align*}
\mathbf{y} &= \begin{bmatrix}
\ln(S_{1}) \\
\ln(S_{2}) \\
\vdots \\
\ln(S_{N})
\end{bmatrix}\\
\end{align*}
$$
The unknown (expected) parameter vector estimate $\hat{\mathbf{\theta}}$ has an analytical solution given by the normal equations:
$$
\boxed{
\begin{align*}
\hat{\mathbf{\theta}} &= (\hat{\mathbf{X}}^{\top}\hat{\mathbf{X}})^{-1}\hat{\mathbf{X}}^{\top}\mathbf{y}\quad\blacksquare\\
\end{align*}}
$$
___

## Stylized Facts
The GBM model is our second method to simulate asset prices. With any model, we want to ensure that it captures the key features of real-world asset prices. Thus, let's ask ourselves: Does the GBM model capture the stylized facts of asset prices?

__Stylized Facts of Asset Prices:__

* __Heavy (also called fat) tailed distribution__: Stock growth rates (returns) often exhibit a distribution with heavier tails than would be expected under a normal distribution. This means that extreme price movements are more likely than would be predicted by a normal distribution.
* __Absence of Autocorrelation__: Autocorrelation refers to the tendency of growth rates (returns) to correlate with past returns over time. Autocorrelation suggests predictability in growth rates (returns), which traders could exploit. On the other hand, a random walk is uncorrelated with itself. We expect a random walk with occasional jumps.
* __Volatility clustering__: Stock growth rates (returns) tend to be more volatile during specific periods and less volatile during others. This phenomenon is known as volatility clustering, suggesting that large price movements are more likely to be followed by other large moves, and other small moves follow small moves.

To start exploring this questions, let's start by computing the growth rates of the GBM model:
$$
\begin{align*}
S_{i} &= S_{0}\;\exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)\left(i\Delta{t}\right) + (\sigma\sqrt{i\Delta{t}})\;{Z}\Biggr]\quad{i = 1,2,\ldots,N}\\
\frac{S_{i}}{S_{0}} &= \exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)\left(i\Delta{t}\right) + (\sigma\sqrt{i\Delta{t}})\;{Z}\Biggr]\\
\ln\left(\frac{S_{i}}{S_{0}}\right) &= \left(\mu-\frac{\sigma^{2}}{2}\right)\left(i\Delta{t}\right) + (\sigma\sqrt{i\Delta{t}})\;{Z}\\
\underbrace{\left(\frac{1}{\Delta{t}}\right)\ln\left(\frac{S_{i}}{S_{0}}\right)}_{= \mu_{i,0}} &= \left(\mu-\frac{\sigma^{2}}{2}\right)i + \sigma\left(\sqrt{\frac{i}{\Delta{t}}}\right)\;{Z}\\  
\mu_{i,0} &= \left(\mu-\frac{\sigma^{2}}{2}\right)i + \sigma\left(\sqrt{\frac{i}{\Delta{t}}}\right)\;{Z}\quad\blacksquare\\
\end{align*}
$$
where $\mu_{i,0}$ is the continuously compounded (multistep) growth rate from time $0\rightarrow{i\Delta{t}}$. 

> __Heavy-Tailed Returns:__ The single step and multistep growth rates $\mu_{i,0}$ are normally distributed with mean $\left(\mu-\frac{\sigma^{2}}{2}\right)i$ and variance $\sigma^{2}\left(\frac{i}{\Delta{t}}\right)$. Thus, the GBM model will not capture the heavy-tailed nature of asset returns. 

## GBM Trade Rule
Let's develop an expression that allows us to compute trade entry and exit rules based on the net present value (NPV) of a trade.

> __Scenario:__ Suppose we purchase $n_{o}$ shares of ticker `XYZ` at time $t=0$ (today) for $S_{0}$ USD/share.
> Then, sometime later, at $T=i\Delta{t}$, we sell all $n_{o}$ shares at a share price of $S_{T}$ USD/share, where $i\geq{0}$ is the number of time steps (integer) and $\Delta{t}$ is the time step (e.g., one trading day written in units of years).


This trade is a __long position__ because we profit if the share price goes up, i.e., $S_{T} > S_{0}$. Further, we can model it as an __abstract asset__ that has two cash flow events: the initial purchase at $t=0$ (entry) and the sale at $t=T$ (exit). The net present value (NPV) of this trade (written from our perspective) will have two cash flow events, the initial purchase at $t=0$ (entry) and the sale at $t=T$ (exit):
$$
\begin{align*}
\texttt{NPV}(\bar{r}, T) &= \underbrace{-n_{o}\;{S_{0}}}_{\text{entry (now)}} + \underbrace{n_{o}\;{S_{T}}\;\mathcal{D}_{T,0}^{-1}(\bar{r})}_{\text{exit (future)}}\quad\Longrightarrow\text{divide by initial investment}\;n_{o}\;{S_{0}}\\
\frac{\texttt{NPV}(\bar{r}, T)}{n_{o}\;{S_{0}}} &= \frac{S_{T}\;\mathcal{D}_{T,0}^{-1}(\bar{r}) - S_{0}}{S_{0}}\\
\underbrace{\frac{\texttt{NPV}(\bar{r}, T)}{n_{o}\;{S_{0}}}}_{\text{fractional return}} &= \left(\frac{S_{T}}{S_{0}}\right)\;\mathcal{D}_{T,0}^{-1}(\bar{r}) - 1\quad\blacksquare
\end{align*}
$$
where $\mathcal{D}^{-1}_{T,0}(\bar{r})$ is the (inverse) continuous discount factor for period $0\rightarrow{T}$ assuming a (constant) discount rate $\bar{r}$. 

Unlike last lecture, we used a binomial lattice models, let's now assume that the share price $S_{T}$ follows a GBM model. Then, we can substitute the GBM solution for $S_{T}$ into our NPV expression and simplify gives:
$$
\boxed{
\begin{align*}
\frac{\texttt{NPV}(\bar{r}, T)}{n_{o}{S_{0}}} &= \exp\Biggl[\left(\mu-\frac{\sigma^{2}}{2}\right)T + (\sigma\sqrt{T})\;{Z(0,1)}\Biggr]\;\mathcal{D}_{T,0}^{-1}(\bar{r}) - 1\quad\blacksquare\\
\end{align*}}
$$

Let's take a closer look at this NPV expression, in the context of short versus long holding periods, our geometric Brownian motion model for the share price $S_{T}$, and postive and negative trade exists.

### Postive Trade Exit Probability
Unlike our binomial lattice model from last lecture, the NPV expression is not a __probability mass function__ (pmf) because the share price $S_{T}$ is a continuous random variable. However, we can still compute the probability that our trade meets some desired fractional return threshold $\texttt{NPV}(\bar{r}, T)/n_{o}S_{0} > r_{\star}$. Let's dig into this a bit more.

Let
$$
X \equiv \left(\mu-\frac{\sigma^{2}}{2}\right)T + \sigma\sqrt{T}\;Z,\qquad Z\sim \mathcal{N}(0,1),
$$
which implies
$$
\frac{\texttt{NPV}(\bar r,T)}{n_o S_0}=e^{X}\mathcal{D}_{T,0}^{-1}(\bar r)-1.
$$
We want the scaled NPV to be (strictly) greater some specified fraction return $r_{\star}>0$, i.e., $\texttt{NPV}(\bar r,T)/n_o S_0> r_\star$. Thus,
$$
\begin{align*}
\mathbb{P}\!\left(\frac{\texttt{NPV}(\bar r,T)}{n_o S_0}> r_\star\right)
& = \mathbb{P}\!\left(e^{X}\mathcal{D}_{T,0}^{-1}(\bar r)-1> r_\star\right)\\
& = \mathbb{P}\!\left(e^{X}> (1+ r_\star)\mathcal{D}_{T,0}(\bar r)\right)\\
& = \mathbb{P}\left(X>\ln\big((1+r_\star)\mathcal{D}_{T,0}(\bar r)\big)\right)
\end{align*}
$$
Ok, so here is the trick to this type of probability calculation. We need to write the probability expression in terms of the cumulative distribution function (CDF) of the standard normal random variable $Z$. To do this, we standardize the random variable $X$. Let $m=(\mu-\tfrac{1}{2}\sigma^2)T$ and $v=\sigma^2 T$. Then, solve $X$ for $Z$:
$$
\mathbb{P}\!\left(X>\ln(1+r_\star)\mathcal{D}_{T,0}(\bar r)\right)
= \mathbb{P}\!\left(\underbrace{\frac{X-m}{\sigma\sqrt{T}}}_{Z\sim\mathcal{N}(0,1)}>\frac{\ln(1+r_\star)\mathcal{D}_{T,0}(\bar r)-m}{\sigma\sqrt{T}}\right)
= 1-\Phi\!\left(\frac{\ln(1+r_\star)\mathcal{D}_{T,0}(\bar r)-m}{\sigma\sqrt{T}}\right),
$$
or equivalently
$$
\boxed{\;
\mathbb{P}\!\left(\frac{\texttt{NPV}(\bar r,T)}{n_o S_0}> r_\star\right)
= 1 - \Phi\!\left(\frac{\ln(1+r_\star)\mathcal{D}_{T,0}(\bar r) - m}{\sigma\sqrt{T}}\right)\quad\blacksquare}
$$

where $\Phi(\cdot)$ is the cumulative distribution for a standard normal random variable. If we hold the $n_{o}$ shares for only a __short time__, e.g., a few days or even a few weeks, for realistic choices of the discount rate $\bar{r}$, the discount factor $\mathcal{D}_{T,0}(\bar{r})\approx{1}$, which simplifies the probability expression.

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.

___